In [ ]:
from transformers import DetrFeatureExtractor, DetrForSegmentation

processor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")


In [ ]:
import io
import requests
from PIL import Image
import torch
import numpy

url = "https://farm4.staticflickr.com/3487/3925656789_1b64654c91_z.jpg"
image = Image.open(requests.get(url, stream=True).raw)


In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
processed_sizes = torch.as_tensor(inputs["pixel_values"].shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes)[0]


In [ ]:
import itertools
import io
import seaborn as sns
import numpy
from transformers.image_transforms import rgb_to_id, id_to_rgb
from matplotlib import pyplot as plt

palette = itertools.cycle(sns.color_palette())
panoptic_seg = Image.open(io.BytesIO(result["png_string"]))
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8).copy()
panoptic_seg_id = rgb_to_id(panoptic_seg)
panoptic_seg[:, :, :] = 0
for id in range(panoptic_seg_id.max() + 1):
    panoptic_seg[panoptic_seg_id == id] = numpy.asarray(next(palette)) * 255
plt.figure(figsize=(15, 15))
plt.imshow(panoptic_seg)
plt.axis("off")
plt.show()
